In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.FIM import FIMModel

In [2]:
hparams = {
    'scale':'demo',
    'name':'fim',
    'batch_size':100,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'epochs':1,
    'k':-1,
    'select':None,
    'save_step':[0],
    'save_each_epoch':True,
    'train_embedding':True,
    'mode':'train',
    'news_id':False,
    'validate':False,
}


In [3]:
vocab, loaders = prepare(hparams)

[2021-03-16 12:48:07,818] INFO (root) Hyper Parameters are
{'scale': 'demo', 'name': 'fim', 'batch_size': 100, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'filter_num': 150, 'embedding_dim': 300, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'epochs': 1, 'k': -1, 'select': None, 'save_step': [0], 'save_each_epoch': True, 'train_embedding': True, 'mode': 'train', 'news_id': False, 'validate': False}
[2021-03-16 12:48:07,819] INFO (root) preparing dataset...
[2021-03-16 12:48:10,142] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(hparams['device'])

In [9]:
train(fimModel, hparams, loaders, interval=10)

[2021-03-16 12:49:57,734] INFO (root) training...
epoch 1 , step 30 , loss: 1.3603: : 32it [00:10,  3.16it/s]
[2021-03-16 12:50:08,732] INFO (root) saved model of epoch 1 at data/model_params/fim/demo_epoch1_step0_[hs=50,topk=-1].model
[2021-03-16 12:50:08,737] INFO (root) evaluating...
186it [00:05, 33.34it/s]
[2021-03-16 12:50:15,430] INFO (root) evaluation results:{'auc': 0.5778, 'mean_mrr': 0.2603, 'ndcg@5': 0.2724, 'ndcg@10': 0.3441, 'step': 0}


FIMModel(
  (softmax): Softmax(dim=-1)
  (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
  (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
  (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  (ReLU): ReLU()
  (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (DropOut): Dropout(p=0.2, inplace=False)
  (SeqCNN3D): Sequential(
    (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
  )
  (learningToRank): Linear(in_features=320, out_features=1, bias=True)
)

In [10]:
evaluate(fimModel, hparams, loaders[1])

[2021-03-16 12:50:26,991] INFO (root) evaluating...
186it [00:05, 32.90it/s]
[2021-03-16 12:50:33,583] INFO (root) evaluation results:{'auc': 0.5778, 'mean_mrr': 0.2603, 'ndcg@5': 0.2724, 'ndcg@10': 0.3441, 'step': 0}


{'auc': 0.5778,
 'mean_mrr': 0.2603,
 'ndcg@5': 0.2724,
 'ndcg@10': 0.3441,
 'step': 0}